<a href="https://colab.research.google.com/github/johir-bd/Machine-Learning-Project/blob/master/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install opencv-python-headless  # Use this if you're running in headless mode (e.g., on a server)

In [22]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2024-12-17 10:39:45--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  16.9MB/s    in 15s     

2024-12-17 10:40:01 (16.3 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [1]:
from google.colab import files
uploaded = files.upload()

Saving 12779573_1280_720_30fps.mp4 to 12779573_1280_720_30fps.mp4


In [2]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Import for displaying images in Colab

# Load the YOLO model
def load_yolo():
    net = cv2.dnn.readNet("/content/yolov3.weights", "/content/yolov3.cfg")  # Update the paths as needed
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

# Load COCO class labels
def load_coco_names():
    with open("/content/coco.names", "r") as f:  # Update the path if needed
        classes = f.read().strip().split("\n")
    return classes

# Perform object detection
def detect_objects(img, net, output_layers):
    height, width = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    return outputs

# Draw bounding boxes on detected objects
def draw_predictions(img, outputs, classes):
    height, width = img.shape[:2]
    boxes = []
    confidences = []
    class_ids = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]  # Scores for classes
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw boxes and labels
    if len(indices) > 0:
        for i in indices.flatten():  # Flatten the indices for easy access
            box = boxes[i]
            x, y, w, h = box
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)  # Green color for bounding boxes
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    else:
        print("No objects detected.")  # Debugging statement

# Main function to run the detection
def main():
    net, output_layers = load_yolo()
    classes = load_coco_names()

    # Load a sample image
    img = cv2.imread('/content/johirul.jpg')  # Change to the path of your image

    if img is None:
        print("Error: Could not read the image.")
        return

    outputs = detect_objects(img, net, output_layers)
    draw_predictions(img, outputs, classes)

    # Use cv2_imshow to display the image in Colab
    cv2_imshow(img)

if __name__ == "__main__":
    main()

error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: /content/yolov3.cfg in function 'readNetFromDarknet'
